Dans cette situation, le défi réside dans la mise en place d'une approche modernisée de Big Data/IA pour la prédiction d'octroi de crédit au sein d'une banque. Le responsable de la ligne de produit crédit à la consommation souhaite prendre des décisions éclairées sur l'attribution des crédits en se basant sur des données antérieures.

La banque dispose d'un ensemble de données comprenant plus de 18 500 crédits, chacun caractérisé par un indicateur et 41 paramètres. L'indicateur, figurant dans la colonne "indicatrice" du fichier de données, distingue les crédits bien remboursés des problématiques (1 indique un problème sur le crédit).

Sur le plan financier, la banque réalise un gain de 4 000 € par crédit bien remboursé, mais subit une perte moyenne de 13 000 € par crédit problématique.

La mission consiste à élaborer une solution Data, accompagnée d'un mode d'emploi, répondant à la demande du responsable de la ligne produit. L'objectif final est d'optimiser l'espérance de bénéfice de la banque dans le processus d'octroi de crédit.

In [92]:
import pandas as pd
CSV_FILE = 'Data_Finance_TrainEtVal_13Nov23.csv'

In [93]:
df = pd.read_csv(CSV_FILE,sep=';')
df.head()

,Id,GENR,produit,age du bien,mode de paiement,montant demande (centimes),delai paiement,apport,apport %,date demande,...,impayes,charge,fichage3,revenus,secteur activite,code postal,Chiffre d'affaires,Capital social,Indicatrice,Label
0,4212339,VP,LLD,0,10,4967310,10,0,0.0,04/01/2010 09:59,...,0,0,0,0,77,95100,0,200000,0,Train
1,4212408,VP,CB,0,10,1228763,0,245753,20.0,04/01/2010 10:58,...,0,0,0,100000,41,93600,0,750000,0,Train
2,4212504,VU,CB,0,10,1262250,0,189338,15.0,04/01/2010 12:44,...,0,0,0,100000,43,47130,0,500000,0,Train
3,4212509,,LLD,0,10,1500000,10,0,0.0,04/01/2010 13:19,...,0,0,0,400000,66,67500,0,1000000,0,Train
4,4212561,VU,LLD,0,10,2393850,10,0,0.0,04/01/2010 14:43,...,0,0,0,300000,49,60110,0,750000,0,Train


In [94]:
for col in df.columns:
  if(df[col].apply(pd.to_numeric, errors='coerce').notna().all()):
    df[col] = (df[col].apply(pd.to_numeric, errors='coerce'))

df.head()

,Id,GENR,produit,age du bien,mode de paiement,montant demande (centimes),delai paiement,apport,apport %,date demande,...,impayes,charge,fichage3,revenus,secteur activite,code postal,Chiffre d'affaires,Capital social,Indicatrice,Label
0,4212339,VP,LLD,0,10,4967310,10,0,0.0,04/01/2010 09:59,...,0,0,0,0,77,95100,0,200000,0,Train
1,4212408,VP,CB,0,10,1228763,0,245753,20.0,04/01/2010 10:58,...,0,0,0,100000,41,93600,0,750000,0,Train
2,4212504,VU,CB,0,10,1262250,0,189338,15.0,04/01/2010 12:44,...,0,0,0,100000,43,47130,0,500000,0,Train
3,4212509,,LLD,0,10,1500000,10,0,0.0,04/01/2010 13:19,...,0,0,0,400000,66,67500,0,1000000,0,Train
4,4212561,VU,LLD,0,10,2393850,10,0,0.0,04/01/2010 14:43,...,0,0,0,300000,49,60110,0,750000,0,Train


In [74]:
categorical_columns = [
    'produit',
    'alerte fraude',
    'Connu',
    'contentieux',
    'fichage1',
    'dirigeant client',
    'fichage2',
    'fichage',
    'incapacite',
    'procédure collective BdF',
    'procédure collective ORT',
    
]
df['Label'] = df['Label'].apply(lambda x: 1 if x == 'Train' else 0).astype('int64')
numerical_columns = [col for col in df.columns if df[col].dtype == 'float64' or df[col].dtype == 'int64']
columns_to_keep = categorical_columns + numerical_columns 
df = df[columns_to_keep]
df.drop('Id')
df.head()


,produit,alerte fraude,Connu,contentieux,fichage1,dirigeant client,fichage2,fichage,incapacite,procédure collective BdF,...,montant impayes internes,age dirigeant,impayes,charge,fichage3,revenus,Chiffre d'affaires,Capital social,Indicatrice,Label
0,LLD,N,N,N,N,N,N,N,,N,...,0,26,0,0,0,0,0,200000,0,1
1,CB,N,N,N,N,N,N,N,,N,...,0,28,0,0,0,100000,0,750000,0,1
2,CB,N,N,N,N,N,N,N,,N,...,0,26,0,0,0,100000,0,500000,0,1
3,LLD,N,N,N,N,O,N,N,,N,...,0,48,0,0,0,400000,0,1000000,0,1
4,LLD,N,N,N,N,N,N,N,,N,...,0,30,0,0,0,300000,0,750000,0,1


In [95]:
df.to_csv('data_clean.csv',index=False)
clean_df = df

In [96]:
clean_df.columns

Index(['Id', 'GENR', 'produit', 'age du bien', 'mode de paiement',
       'montant demande (centimes)', 'delai paiement', 'apport', 'apport %',
       'date demande', 'duree', 'NO_FOUR', 'cotation', 'code naf',
       'alerte fraude', 'Connu', 'fichage', 'incapacite', 'segmentation',
       'existence impaye regularise', 'presence homonyme', 'Montant',
       'effets impayes', 'montant impayes internes',
       'procédure collective BdF', 'procédure collective ORT', 'age dirigeant',
       'contentieux', 'CSP dirigeant', 'cotation dirigeant', 'fichage1',
       'dirigeant client', 'fichage2', 'impayes', 'charge', 'fichage3',
       'revenus', 'secteur activite', 'code postal', 'Chiffre d'affaires',
       'Capital social', 'Indicatrice', 'Label'],
      dtype='object')

In [97]:
one_hot_encoded_df = pd.get_dummies(clean_df, columns=categorical_columns)

In [98]:
X_train = one_hot_encoded_df[one_hot_encoded_df['Label'] == 1].drop(['Label','Indicatrice'], axis=1)
X_test = one_hot_encoded_df[one_hot_encoded_df['Label'] == 0].drop(['Label','Indicatrice'], axis=1)
y_train = clean_df[clean_df['Label'] == 1]['Indicatrice']
y_test = clean_df[clean_df['Label'] == 0]['Indicatrice']

In [99]:
# Apply random forest
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=100, random_state=0)
random_forest.fit(X_train, y_train)
# Predict
y_pred = random_forest.predict(X_test)
# Evaluate
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)
# Evaluate other metrics
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)
# plot confusion matrix
import matplotlib.pyplot as plt
import seaborn as sns
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Truth')
# plot feature importance
plt.figure(figsize=(10,10))
plt.barh(X_train.columns, random_forest.feature_importances_)
plt.xlabel('Importance')
plt.ylabel('Feature')

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


ValueError: Found array with 0 sample(s) (shape=(0, 59)) while a minimum of 1 is required by RandomForestClassifier.